# Sesión 02 - Biopython: trabajo con ficheros FASTA

#### **Autores: Adrián Ojeda Viera y Raúl Mendoza Peña**

Este cuaderno resuelve el ejercicio propuesto en la práctica **"Bioinformática en laboratorio - Sesión 02"**.

El objetivo es:
1. Leer un fichero FASTA con varias secuencias de ADN.
2. Calcular el contenido de GC (en porcentaje) para cada secuencia y mostrarlo por pantalla.
3. Traducir cada secuencia de ADN a su cadena de aminoácidos.
4. Guardar las secuencias de aminoácidos en un nuevo fichero FASTA, usando los mismos identificadores y descripciones que en el archivo original.

## 1. Preparación del entorno

- Es necesario tener instalado **Biopython**.
  ```bash
  pip install biopython
  ```
- Por defecto, el archivo de entrada se llamará `gene.fna`, pero puedes cambiarlo en la celda de abajo, si desea utilizar otro archivo.

In [1]:
from pathlib import Path

input_fasta = Path('gene.fna')  
output_fasta = Path('proteinas_traducidas.fasta')

print('Archivo de entrada:', input_fasta.resolve())
print('Archivo de salida :', output_fasta.resolve())

Archivo de entrada: C:\Users\adrio\OneDrive\Escritorio\ULPGC\CUARTO_AÑO\BIOINFORMÁTICA\bioinformatica-trabajoconficheros\gene.fna
Archivo de salida : C:\Users\adrio\OneDrive\Escritorio\ULPGC\CUARTO_AÑO\BIOINFORMÁTICA\bioinformatica-trabajoconficheros\proteinas_traducidas.fasta


## 2. Importar Biopython y definir función para el contenido GC

La función `gc_content` recibirá una secuencia de ADN y devolverá el porcentaje de nucleótidos **G** y **C**.

In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

def gc_content(seq: Seq) -> float:
    seq_str = str(seq).upper()
    if len(seq_str) == 0:
        return 0.0
    g = seq_str.count('G')
    c = seq_str.count('C')
    gc = (g + c) / len(seq_str) * 100   
    return gc


## 3. Leer el archivo FASTA y calcular el contenido GC

En esta celda:
- Leemos todas las secuencias del fichero FASTA.
- Calculamos el contenido GC de cada una.
- Mostramos por pantalla el identificador y el porcentaje de GC.

In [3]:
if not input_fasta.exists():
    raise FileNotFoundError(f'No se encontró el archivo de entrada: {input_fasta}')

records = list(SeqIO.parse(str(input_fasta), 'fasta'))

if not records:
    raise ValueError('El archivo FASTA no contiene ninguna secuencia.')

print('Contenido GC por secuencia:\n')
for record in records:
    gc = gc_content(record.seq)
    print(f"ID: {record.id}\tGC: {gc:.2f}%")

Contenido GC por secuencia:

ID: NC_000012.12:c47904994-47841537	GC: 49.37%
ID: NC_060936.1:c47866398-47802918	GC: 49.34%


## 4. Traducir las secuencias de ADN a aminoácidos

A continuación traducimos cada secuencia de ADN a su correspondiente secuencia de aminoácidos usando el código genético estándar.

Detalles de la implementación:
- Se utiliza el método `translate` de Biopython.
- El parámetro `to_stop=True` traduce hasta el primer codón de parada.
- Si tu secuencia no está en marco de lectura correcto, puede que quieras ajustar manualmente el *frame*.

Guardaremos las secuencias traducidas en una lista de `SeqRecord` llamada `protein_records`, que luego utilizaremos para escribir el archivo FASTA de salida.

In [4]:
protein_records = []

for record in records:
    protein_seq = record.seq.translate(to_stop=True)

    protein_record = SeqRecord(
        protein_seq,
        id=record.id,  
        description=record.description  
    )
    protein_records.append(protein_record)

print(f'Se han generado {len(protein_records)} secuencias de aminoácidos.')
print('Ejemplo del primer registro traducido:')
print(protein_records[0].format('fasta'))

Se han generado 2 secuencias de aminoácidos.
Ejemplo del primer registro traducido:
>NC_000012.12:c47904994-47841537 VDR [organism=Homo sapiens] [GeneID=7421] [chromosome=12]
GSAEQLVHPPAGPGANPGAAGKGVSG



c:\Users\adrio\OneDrive\Escritorio\ULPGC\CUARTO_AÑO\BIOINFORMÁTICA\PRACTCAS_JOSECARLOS\PRACTICA2_TRABAJANDOCONFICHEROS\.venv\Lib\site-packages\Bio\Seq.py:2877: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


## 5. Escribir las secuencias de aminoácidos en un archivo FASTA

Finalmente, escribimos todas las secuencias de proteínas en un nuevo fichero FASTA llamado `proteinas_traducidas.fasta`. Los identificadores (`id`) y descripciones (`description`) se conservan de los registros originales.

In [ ]:
num_written = SeqIO.write(protein_records, str(output_fasta), 'fasta')
print(f'Se han escrito {num_written} secuencias en el archivo: {output_fasta}')

## 6. Resumen

En este cuaderno se realiza con exito los pasos pedidos en el ejercicio:
1. Leer un fichero FASTA con varias secuencias de ADN.
2. Calcular y mostrar el contenido GC de cada una.
3. Traducir las secuencias de ADN a aminoácidos.
4. Guardar las secuencias de aminoácidos en un nuevo archivo FASTA conservando identificadores y descripciones.